<a href="https://colab.research.google.com/github/audwls4545/metaverse/blob/main/species_of_tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.datasets import fetch_covtype

In [ ]:
#다음은 미국 삼림을 30M*30M 로 나누어서 각 영역의 특징으로부터 대표적인 나무의 종류를예측하기 위한 데이터입니다.
#대표나무 species of tree를 예측하는 모델을 만드세요.
#random state 값은 100으로 통일

In [ ]:
data = fetch_covtype()

In [ ]:
import xgboost as xgb
from xgboost import plot_importance
import pandas as pd
import seaborn as sns
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

features= data.data
labels = data.target

In [ ]:
tree_df = pd.DataFrame(data = features, columns = data.feature_names)
tree_df['target'] = labels
tree_df.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type_31,Soil_Type_32,Soil_Type_33,Soil_Type_34,Soil_Type_35,Soil_Type_36,Soil_Type_37,Soil_Type_38,Soil_Type_39,target
0,2596.0,51.0,3.0,258.0,0.0,510.0,221.0,232.0,148.0,6279.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
1,2590.0,56.0,2.0,212.0,-6.0,390.0,220.0,235.0,151.0,6225.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
2,2804.0,139.0,9.0,268.0,65.0,3180.0,234.0,238.0,135.0,6121.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
3,2785.0,155.0,18.0,242.0,118.0,3090.0,238.0,238.0,122.0,6211.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
4,2595.0,45.0,2.0,153.0,-1.0,391.0,220.0,234.0,150.0,6172.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5


In [ ]:
tree_df['target'].value_counts()

2    283301
1    211840
3     35754
7     20510
6     17367
5      9493
4      2747
Name: target, dtype: int64

In [ ]:
tree_df['target'].unique()

array([5, 2, 1, 7, 3, 6, 4], dtype=int32)

In [ ]:
tree_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581012 entries, 0 to 581011
Data columns (total 55 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   Elevation                           581012 non-null  float64
 1   Aspect                              581012 non-null  float64
 2   Slope                               581012 non-null  float64
 3   Horizontal_Distance_To_Hydrology    581012 non-null  float64
 4   Vertical_Distance_To_Hydrology      581012 non-null  float64
 5   Horizontal_Distance_To_Roadways     581012 non-null  float64
 6   Hillshade_9am                       581012 non-null  float64
 7   Hillshade_Noon                      581012 non-null  float64
 8   Hillshade_3pm                       581012 non-null  float64
 9   Horizontal_Distance_To_Fire_Points  581012 non-null  float64
 10  Wilderness_Area_0                   581012 non-null  float64
 11  Wilderness_Area_1         

In [ ]:
q3 = tree_df['target'].quantile(0.75)
q1 = tree_df['target'].quantile(0.25)

iqr = q3 - q1

In [ ]:
def is_tree_outlier(tree_df):
    score = tree_df['target']
    if score > q3 + 1.5 * iqr or score < q1 - 1.5 * iqr:
        return True
    else:
        return False

tree_df['target_error'] = tree_df.apply(is_tree_outlier, axis = 1) # axis = 1 지정 필수

tree_df['target_error']

0          True
1          True
2         False
3         False
4          True
          ...  
581007    False
581008    False
581009    False
581010    False
581011    False
Name: target_error, Length: 581012, dtype: bool

In [ ]:
# 이상치인 행은 제거하여 필터링
tree_df = tree_df.loc[tree_df['target_error'] == False]

# 이상치여부를 나타내는 열 제거
del tree_df['target_error']

tree_df

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type_31,Soil_Type_32,Soil_Type_33,Soil_Type_34,Soil_Type_35,Soil_Type_36,Soil_Type_37,Soil_Type_38,Soil_Type_39,target
2,2804.0,139.0,9.0,268.0,65.0,3180.0,234.0,238.0,135.0,6121.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
3,2785.0,155.0,18.0,242.0,118.0,3090.0,238.0,238.0,122.0,6211.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
5,2579.0,132.0,6.0,300.0,-15.0,67.0,230.0,237.0,140.0,6031.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
11,2886.0,151.0,11.0,371.0,26.0,5253.0,234.0,240.0,136.0,4051.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
12,2742.0,134.0,22.0,150.0,69.0,3215.0,248.0,224.0,92.0,6091.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581007,2396.0,153.0,20.0,85.0,17.0,108.0,240.0,237.0,118.0,837.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
581008,2391.0,152.0,19.0,67.0,12.0,95.0,240.0,237.0,119.0,845.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
581009,2386.0,159.0,17.0,60.0,7.0,90.0,236.0,241.0,130.0,854.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
581010,2384.0,170.0,15.0,60.0,5.0,90.0,230.0,245.0,143.0,864.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3


In [ ]:
tree_df['target'].value_counts()

2    283301
1    211840
3     35754
Name: target, dtype: int64

In [ ]:
from sklearn.tree import DecisionTreeClassifier

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=3)
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)
print('모델의 정확도 :', round(dtc.score(X_test, y_test), 4))

모델의 정확도 : 0.9396


In [ ]:
from sklearn.preprocessing import StandardScaler

std = StandardScaler()
std.fit(X_train)
X_train_scaled = std.transform(X_train)
X_test_scaled = std.transform(X_test)
dtc.fit(X_train_scaled, y_train)
print('모델의 정확도 :', round(dtc.score(X_test_scaled, y_test), 4))

모델의 정확도 : 0.9395


In [ ]:
from sklearn.preprocessing import MinMaxScaler

mms = MinMaxScaler()
mms.fit(X_train)
X_train_scaled = mms.transform(X_train)
X_test_scaled = mms.transform(X_test)
dtc.fit(X_train_scaled, y_train)
print('모델의 정확도 :', round(dtc.score(X_test_scaled, y_test), 4))

모델의 정확도 : 0.9401


In [ ]:
from sklearn.preprocessing import MaxAbsScaler

mas = MaxAbsScaler()
mas.fit(X_train)
X_train_scaled = mas.transform(X_train)
X_test_scaled = mas.transform(X_test)
dtc.fit(X_train_scaled, y_train)
print('모델의 정확도 :', round(dtc.score(X_test_scaled, y_test), 4))

모델의 정확도 : 0.9401


In [ ]:
from sklearn.preprocessing import RobustScaler

rbs = RobustScaler()
X_train_scaled = rbs.fit_transform(X_train)
X_test_scaled = rbs.transform(X_test)
dtc.fit(X_train_scaled, y_train)
print('모델의 정확도 :', round(dtc.score(X_test_scaled, y_test), 4))

모델의 정확도 : 0.9396


In [ ]:
from sklearn.preprocessing import Normalizer

norm = Normalizer()
X_train_scaled = norm.fit_transform(X_train)
X_test_scaled = norm.transform(X_test)
dtc.fit(X_train_scaled, y_train)
print('모델의 정확도 :', round(dtc.score(X_test_scaled, y_test), 4))

모델의 정확도 : 0.9322


In [ ]:
tree_df['target'].value_counts()

2    283301
1    211840
3     35754
Name: target, dtype: int64

In [ ]:
tree_df['target'].unique()

array([2, 1, 3], dtype=int32)

In [ ]:
X_features = tree_df.iloc[:,:-1]
y_label = tree_df.iloc[:,-1]
y_label

2         2
3         2
5         2
11        2
12        2
         ..
581007    3
581008    3
581009    3
581010    3
581011    3
Name: target, Length: 530895, dtype: int32

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_features, y_label, test_size = 0.2,
                                                    random_state = 156)

X_tr, X_val, y_tr, y_val= train_test_split(X_train, y_train, test_size=0.1, random_state=156 )
print(X_train.shape , X_test.shape)
print(X_tr.shape, X_val.shape)

(424716, 54) (106179, 54)
(382244, 54) (42472, 54)


In [ ]:
dtr = xgb.DMatrix(data=X_tr, label=y_tr)
dval = xgb.DMatrix(data=X_val, label=y_val)
dtest = xgb.DMatrix(data=X_test , label=y_test)

In [ ]:
params = { 'max_depth':3,
          'eta': 0.05,
          'objective':'reg:linear',
          'eval_metric':'logloss'
         }
num_rounds = 400

# 학습 데이터 셋은 'train' 또는 평가 데이터 셋은 'eval' 로 명기합니다. 
eval_list = [(dtr,'train'),(dval,'eval')] # 또는 eval_list = [(dval,'eval')] 만 명기해도 무방. 

# 하이퍼 파라미터와 early stopping 파라미터를 train( ) 함수의 파라미터로 전달
xgb_model = xgb.train(params = params , dtrain=dtr , num_boost_round=num_rounds ,
                      early_stopping_rounds=50, evals=eval_list )

[07:08:13] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	train-logloss:0.39263	eval-logloss:0.39141
[1]	train-logloss:0.10725	eval-logloss:0.10470
[2]	train-logloss:-0.18142	eval-logloss:-0.18567
[3]	train-logloss:-0.52069	eval-logloss:-0.52872
[4]	train-logloss:-3.66025	eval-logloss:-3.78296
[5]	train-logloss:-4.56926	eval-logloss:-4.70358
[6]	train-logloss:-5.90695	eval-logloss:-5.98800
[7]	train-logloss:-8.34230	eval-logloss:-8.35864
[8]	train-logloss:-9.32843	eval-logloss:-9.35643
[9]	train-logloss:-17.63733	eval-logloss:-17.72970
[10]	train-logloss:-19.40842	eval-logloss:-19.48874
[11]	train-logloss:-20.05180	eval-logloss:-20.10698
[12]	train-logloss:-20.47937	eval-logloss:-20.52335
[13]	train-logloss:-20.87634	eval-logloss:-20.92057
[14]	train-logloss:-21.28109	eval-logloss:-21.31942
[15]	train-logloss:-21.76422	eval-logloss:-21.80437
[16]	train-logloss:-22.50991	eval-logloss:-22.54392
[17]	train-logloss:-23.15131	

In [ ]:
pred_probs = xgb_model.predict(dtest)
print('predict( ) 수행 결과값을 표시, 예측 확률 값으로 표시됨')
pred_probs

predict( ) 수행 결과값을 표시, 예측 확률 값으로 표시됨


array([1.8811834, 1.1004764, 1.3894609, ..., 1.8673991, 1.3574008,
       1.8928337], dtype=float32)

In [ ]:
pred_probs = xgb_model.predict(dtest)
print('predict( ) 수행 결과값을 10개만 표시, 예측 확률 값으로 표시됨')
print(np.round(pred_probs[:10],3))

preds = [ 3 if x >2.5 else 2 if x > 1.5 else 1 if x > 0.5 else 0 for x in pred_probs ]
#preds = [ 1 if x > 0.5 else 0 for x in pred_probs ]
print('예측값 10개만 표시:',preds[:10])

predict( ) 수행 결과값을 10개만 표시, 예측 확률 값으로 표시됨
[1.881 1.1   1.389 2.223 1.803 2.624 1.975 2.026 1.884 1.079]
예측값 10개만 표시: [2, 1, 1, 2, 2, 3, 2, 2, 2, 1]


In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.metrics import fbeta_score # f베타 스코어

In [ ]:
confusion_matrix(y_test, preds)

array([[31820, 10424,     2],
       [ 9605, 46427,   653],
       [    0,   780,  6468]])

In [ ]:
# 정밀도(precision)
pre_micro = precision_score(y_test, preds, average = 'micro')
print(pre_micro) #정확도 값과 동일

pre_macro = precision_score(y_test, preds, average = 'macro')
print(pre_macro)

pre_weighted = precision_score(y_test, preds, average = 'weighted')
print(pre_weighted)

# 재현율(recall)
recall_micro = recall_score(y_test, preds, average= 'micro')
print(float(recall_micro)) #정확도 값과 동일

recall_macro = recall_score(y_test, preds, average= 'macro')
print(recall_macro)

recall_weighted = recall_score(y_test, preds, average = 'weighted')
print(recall_weighted)

# F1 score
f1_micro = f1_score(y_test, preds, average= 'micro')
print(f1_micro) #정확도 값과 동일

f1_macro = f1_score(y_test, preds, average= 'macro')
print(f1_macro)

f1_weighted = f1_score(y_test, preds, average = 'weighted')
print(f1_weighted)

# Fbeta score (beta = 2)
fbeta_micro = fbeta_score(y_test, preds, beta = 2, average= 'micro')
print(fbeta_micro)#정확도 값과 동일

fbeta_macro = fbeta_score(y_test, preds, beta = 2, average= 'macro')
print(fbeta_macro)

fbeta_weighted = fbeta_score(y_test, preds, beta = 2, average = 'weighted')
print(fbeta_weighted)

accuracy = accuracy_score(y_test, preds, average = 'weighted')
print(accuracy)

0.797850799122237
0.8272567628306174
0.7976817985709098
0.797850799122237
0.8215421760979801
0.797850799122237
0.797850799122237
0.8243338120402469
0.7977028127513556
0.797850799122237
0.8226431803549675
0.7977763976690238


TypeError: ignored

In [ ]:
# 모델 저장
import joblib
from google.colab import files
import time

joblib.dump(model, 'model.pkl')
time.sleep(10)
files.download('model.pkl')

In [ ]:
# 모델 불러오기
clf_from_joblib = joblib.load('model.pkl')
clf_from_joblib.predict(X)